**Loading and Inspecting Census Data with Pandas**

In [2]:
import pandas as pd

# Load the updated census data
census_data = pd.read_csv("kenya-population-by-sex-and-county.csv")

# Inspect the data
print(census_data.head())


       County  Male_Population  Female_Population  Intersex_Population  \
0     Mombasa           610257             598046                   30   
1       Kwale           425121             441681                   18   
2      Kilifi           704089             749673                   25   
3  Tana River           158550             157391                    2   
4        Lamu            76103              67813                    4   

   Total_Population  
0           1208333  
1            866820  
2           1453787  
3            315943  
4            143920  


**Loading, Merging, and Inspecting Kenyan Census Data with Geospatial Information**

In [21]:
import geopandas as gpd

# Load the shapefile for Kenyan counties (Level 1)
shapefile_path = "C:/Users/manubungei/Desktop/Data Kaggle/2019 Kenya Population and Housing Census Datasets/Kenyan_Counties/gadm41_KEN_1.shp"  
gdf = gpd.read_file(shapefile_path)

# Standardize the names in both datasets (lowercase and strip)
gdf['NAME_1'] = gdf['NAME_1'].str.lower().str.strip()
census_data['County'] = census_data['County'].str.lower().str.strip()

# Merge the shapefile with the census data
merged_gdf = gdf.merge(census_data, left_on="NAME_1", right_on="County", how="left")

# Check for missing data
print(merged_gdf[merged_gdf['Total_Population'].isnull()])


Empty GeoDataFrame
Columns: [GID_1, GID_0, COUNTRY, NAME_1, VARNAME_1, NL_NAME_1, TYPE_1, ENGTYPE_1, CC_1, HASC_1, ISO_1, geometry, County, Male_Population, Female_Population, Intersex_Population, Total_Population]
Index: []



**Counting Missing Population Data in Merged Geospatial Dataset**

In [6]:
merged_gdf['Total_Population'].isna().sum()  # Count NaN values in the population column


0

**Creating an Interactive Map of Kenyan Population Data with Folium and GeoJSON**

In [8]:
import folium
from folium.features import GeoJsonTooltip
import branca

# Check for missing data in the merged GeoDataFrame
taita_taveta = merged_gdf[merged_gdf['NAME_1'] == 'Taita Taveta']
print("Taita Taveta Data:", taita_taveta)

# Define a function to determine color based on total population using a continuous color scale
# Using LinearColormap from branca for more detailed color gradation, with a maximum of 3 million
colormap = branca.colormap.linear.YlOrRd_09.scale(0, 3000000)  # Max value set to 3 million

# Add a default value for missing population data
def get_population(population):
    return population if population is not None else 0  # Handle None values

# Create a Folium map centered on Kenya
m = folium.Map(location=[0.0236, 37.9062], zoom_start=6)

# Add GeoJSON layer with tooltip including Total Population and other population data
folium.GeoJson(
    merged_gdf,
    style_function=lambda x: {
        'fillColor': colormap(get_population(x['properties']['Total_Population'])),  # Handle None values
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=GeoJsonTooltip(
        fields=["NAME_1", "Male_Population", "Female_Population", "Intersex_Population", "Total_Population"],
        aliases=["County:", "Male Population:", "Female Population:", "Intersex Population:", "Total Population:"],
        localize=True,
        sticky=False,
        labels=True,
        style=("background-color: white; color: black; font-family: arial; font-size: 12px; padding: 5px;"),
    ),
    highlight_function=lambda x: {
        'weight': 3,
        'color': 'black',
        'fillOpacity': 0.9,
        'fillColor': 'blue',  # Enlarge and highlight in blue on hover
    }
).add_to(m)

# Add the colormap to the map as a legend
colormap.add_to(m)

# Save the map
m.save("kenya_population_colored_with_total_population_map.html")
print("Map saved as 'kenya_population_colored_with_total_population_map.html'")


Taita Taveta Data: Empty GeoDataFrame
Columns: [GID_1, GID_0, COUNTRY, NAME_1, VARNAME_1, NL_NAME_1, TYPE_1, ENGTYPE_1, CC_1, HASC_1, ISO_1, geometry, County, Male_Population, Female_Population, Intersex_Population, Total_Population]
Index: []
Map saved as 'kenya_population_colored_with_total_population_map.html'


**Customizing Tooltips for GeoJSON Layer in Folium Map**

In [10]:
tooltip=GeoJsonTooltip(
    fields=["NAME_1", "Male_Population", "Female_Population", "Intersex_Population", "Total_Population"],
    aliases=["County:", "Male Population:", "Female Population:", "Intersex Population:", "Total Population:"],
    localize=True,
    sticky=False,
    labels=True,
    style=("background-color: white; color: black; font-family: arial; font-size: 12px; padding: 5px;"),
)
